In [14]:
import tensorflow as tf
import importlib
from model.model import Unet,Unet_Model
import model.model
importlib.reload(model.model)
try: 
    del unet
except:
    pass



In [16]:
unet = Unet(n_classes=4)
unet.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
unet.model((128,128,1)).summary()
 

5
4
(None, 8, 8, 512)
(None, 16, 16, 256) (None, 16, 16, 256)
(None, 16, 16, 256)
(None, 32, 32, 128) (None, 32, 32, 128)
(None, 32, 32, 128)
(None, 64, 64, 64) (None, 64, 64, 64)
(None, 64, 64, 64)
(None, 128, 128, 32) (None, 128, 128, 32)
Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 128, 128, 1  0           []                               
                                )]                                                                
                                                                                                  
 downsampling_block_35 (Downsam  ((None, 64, 64, 32)  9568       ['input_3[0][0]']                
 plingBlock)                    , (None, 128, 128,                                                
                                32))             

In [18]:
from dataloader.dataloader import Series,Serie
from configs.config import CFG
import pandas as pd
import numpy as np


NameError: name 'Series' is not defined

In [ ]:
import importlib
import dataloader.dataloader
from dataloader.dataloader import Series,Serie,Dataset
importlib.reload(dataloader.dataloader)
try: 
    del series
except:
    pass

from configs.config import CFG
from utils import config
config = config.Config.from_json(CFG)

In [ ]:
series = Series(config.data,config.paths)
series.createSeries()
ds = Dataset(series)
ds.split_dataset()
